In [ ]:
########################################
import os
import re
import json
import random
import pickle
import operator
import itertools

import numpy
import sklearn.ensemble
import sklearn.linear_model
import sklearn.feature_extraction
import pandas
import html
#import raha
########################################

In [ ]:
DATASET_CONSTRAINTS = {
            "hospital": {
                "functions": [["city", "zip"], ["city", "county"], ["zip", "city"], ["zip", "state"], ["zip", "county"],
                              ["county", "state"]]
               
            },
            "flights": {
                "functions": [["flight", "act_dep_time"], ["flight", "sched_arr_time"], ["flight", "act_arr_time"],
                              ["flight", "sched_dep_time"]]
            
            },
    "tax":{
        
        "functions":[['zip','city'],['zip','state'],['area_code','state']]
    }
}
    


In [116]:
dataset_name = "tax"
dataset_dictionary = {
        "name": dataset_name,
        "path": os.path.abspath(os.path.join( "datasets", dataset_name, "dirty.csv")),
        "clean_path": os.path.abspath(os.path.join("datasets", dataset_name, "clean.csv"))
    }

In [117]:
def value_normalizer(value):
        """
        This method takes a value and minimally normalizes it.
        """
        value = html.unescape(value)
        value = re.sub("[\t\n ]+", " ", value, re.UNICODE)
        value = value.strip("\t\n ")
        return value

In [118]:
def read_csv_dataset(dataset_path):
        """
        This method reads a dataset from a csv file path.
        """
        dataframe = pandas.read_csv(dataset_path, sep=",", header="infer", encoding="utf-8", dtype=str,
                                    keep_default_na=False, low_memory=False).applymap(value_normalizer)
        return dataframe


In [119]:
name = dataset_dictionary["name"]
path = dataset_dictionary["path"]
dataframe = read_csv_dataset(dataset_dictionary["path"])
if "clean_path" in dataset_dictionary:
    has_ground_truth = True
    clean_path = dataset_dictionary["clean_path"]
    clean_dataframe = read_csv_dataset(dataset_dictionary["clean_path"])
if "repaired_path" in dataset_dictionary:
    has_been_repaired = True
    repaired_path = dataset_dictionary["repaired_path"]
    repaired_dataframe = read_csv_dataset(dataset_dictionary["repaired_path"])


In [120]:
total_error=calculate_total_error_realworld(clean_dataframe, dataframe)

In [121]:
total_error

121219

In [122]:
# library for fasttext
import warnings
warnings.filterwarnings('ignore')
import fasttext
from fasttext import train_unsupervised
import gensim
from gensim.models import FastText
import ast
########################################
#library for parsing and extracting wiki revision data
import os
import re
import io
import sys
import math
import json
import html
import pickle
import difflib
import unicodedata
import bs4
import bz2
import py7zr
import numpy
import mwparserfromhell
import libarchive.public
#import dataset
import wikitextparser as wtp
from pprint import pprint
from wikitextparser import remove_markup, parse
import datetime
from pandas import DataFrame
import random
import pickle
import pandas as pd
import logging
from datasets import *
import pandas

In [123]:
flight_fds=[['flight','act_dep_time'],['flight','sched_dep_time'],['flight','sched_arr_time'],['flight','act_arr_time']]
flight_fds_att=['flight','act_dep_time','sched_dep_time','sched_arr_time','act_arr_time']
domain_dirty_col=['address_1','city','state','county','state','area_code']
domain_clean_col=['Address1','City','State','CountyName','city','state','area_code']


In [151]:
flight_fds=[['city','state','area_code']]

In [152]:
retrain_data=prepare_dataset_for_retrain_realworld(clean_dataframe, dataframe)


In [85]:
#retrain_data[['flight', 'act_dep_time']]
from gensim.models import Word2Vec

In [153]:
score=error_correction_fasttext_with_retrain_realworld_fds_new("Fasttext_D_F",retrain_data,"flights",flight_fds,total_error, dataframe, clean_dataframe,domain_dirty_col,domain_clean_col)
print(score)

['city', 'state', 'area_code']
0 800
121219 800 0
(0.0, 0.0, 0.0)


In [141]:
score #try with whole attributes

(0.0, 0.0, 0.0)

In [67]:
from gensim.models import FastText

In [73]:
model = FastText(size=4, window=3, min_count=1)

In [72]:
gensim.__version__

'3.8.0'

In [150]:
def error_correction_fasttext_with_retrain_realworld_fds_new(model_type,data_for_retrain,dataset_name,fds_set,total_error, dirty_data, clean_data,domain_dirty_col,domain_clean_col):
        #total_error=0
        total_error_to_repaired=0
        total_repaired=0
        dirty_data=data_for_retrain
        try:
            if model_type=="Fasttext_G_F":
                model_fasttext=FastText.load("model/Fasttext_All_Domain.w2v")
            if model_type=="Fasttext_D_F":
                model_fasttext=FastText.load("model/Fasttext_Location_Domain.w2v")
        except Exception as e:
            print('Model Error: ',str(e))
        err_fds_con=[]
        for fds in  fds_set:
            print(fds)
            data_for_retrain=dirty_data #retrain the whole datasest
            train_data_rows=[]
            try:         
                data_for_retrain=data_for_retrain.values.tolist()
                for row in data_for_retrain:
                    #row = list(map(str, row))
                    row=list(filter(None, row))
                    if 'None' in row:
                        continue
                    else:
                        train_data_rows.append(row)
                if train_data_rows:
                   # print(train_data_rows)
                    if train_data_rows:
                        model_fasttext = FastText(train_data_rows, min_count=1, workers=8, iter=500, window=len(train_data_rows[0]))
                        #model = Word2Vec(train_data_rows, sg=1, min_count=1, workers=8, iter=1000, window=2)
                        #model_fasttext.build_vocab(train_data_rows, update=True)
                        #model_fasttext=(vector_size=4, window=3, min_count=1, sentences=train_data_rows, epochs=10)
                        #model_fasttext.train(sentences=train_data_rows, total_examples = len(train_data_rows), epochs=5)
            except Exception as e:
                print("Exception from spell model : ", str(e))
            err_fds=[]
            for col_fds in fds:
                if col_fds in err_fds_con:
                    continue
                else:
                    err_fds_con.append(col_fds)
                    err_fds.append(col_fds)
            if model_type=="Fasttext_G_F":
                error_correction=prepare_testing_datasets_real_world_data_error(dirty_data,clean_data,"fds",dataset_name,fds)
            else:
                error_correction=prepare_domain_testing_datasets_real_world_data_error(dirty_data,clean_data,"fds",dataset_name,domain_dirty_col,domain_clean_col,fds)
            for error_value, actual_value,want_to_clean,index in zip(error_correction['error'],error_correction['actual'],error_correction['want_to_clean'],error_correction['index']):
                try:
                    dirty_row=[]
                    if fds:
                        for fds_col in fds:
                            dirty_row.append(str(dirty_data.at[index, fds_col]))
                    if dirty_row:
                        #dirty_row = list(map(str, dirty_row))
                        dirty_row=list(filter(None, dirty_row))
                        if error_value:
                            dirty_row.remove(error_value)
                    #total_error=total_error+1
                    want_to_clean=str(want_to_clean)
                   # print(dirty_row, error_value)
                    if want_to_clean=="1" and len(error_value)<20 and total_error_to_repaired<10000:
                        total_error_to_repaired=total_error_to_repaired+1
                        if fds and dirty_row:
                            similar_value=model_fasttext.most_similar(positive=dirty_row, negative=[error_value])
                            #similar_value=model_fasttext.most_similar(dirty_row[0])
                            #R_Value=model.wv.most_similar(positive=dirty_row)
                        else:
                            similar_value=model_fasttext.most_similar(error_value)
                        #print(similar_value)
                        #R_Value=model.wv.most_similar(positive=dirty_row)
                        #print(R_Value)
                        #a=R_Value[0]
                        #b,c=a
                        #first=b
                        first,b=similar_value[0]
                        actual_value=str(actual_value)               
                        first=first.strip()
                        actual_value=actual_value.strip()
                        #print(first,actual_value)
                        if first==actual_value:
                            total_repaired=total_repaired+1
                except Exception as e:
                    print('Error correction model: ',str(e))
            print(total_repaired,total_error_to_repaired)
        print(total_error,total_error_to_repaired,total_repaired )
        if total_error_to_repaired>0:
            p,r,f=evaluate_model(total_error,total_error_to_repaired,total_repaired)
        else:
            p,r,f="Invalid", "Invalid", "Invalid"
        return p,r,f

In [148]:
def ar_error_correction_fasttext_with_retrain_realworld_fds_new(model_type,data_for_retrain,dataset_name,fds_set,total_error, dirty_data, clean_data,domain_dirty_col,domain_clean_col):
        #total_error=0
        total_error_to_repaired=0
        total_repaired=0
        dirty_data=data_for_retrain
        try:
            if model_type=="Fasttext_G_F":
                model_fasttext=FastText.load("model/Fasttext_All_Domain.w2v")
            if model_type=="Fasttext_D_F":
                model_fasttext=FastText.load("model/Fasttext_Location_Domain.w2v")
        except Exception as e:
            print('Model Error: ',str(e))
        err_fds_con=[]
        for fds in  fds_set:
            print(fds)
            data_for_retrain=dirty_data #retrain the whole datasest
            train_data_rows=[]
            try:         
                data_for_retrain=data_for_retrain.values.tolist()
                for row in data_for_retrain:
                    #row = list(map(str, row))
                    row=list(filter(None, row))
                    if 'None' in row:
                        continue
                    else:
                        train_data_rows.append(row)
                if train_data_rows:
                   # print(train_data_rows)
                    if train_data_rows:
                        model_fasttext = FastText(train_data_rows, min_count=1, workers=8, iter=500, window=len(train_data_rows[0]))
                        #model = Word2Vec(train_data_rows, sg=1, min_count=1, workers=8, iter=1000, window=2)
                        #model_fasttext.build_vocab(train_data_rows, update=True)
                        #model_fasttext=(vector_size=4, window=3, min_count=1, sentences=train_data_rows, epochs=10)
                        #model_fasttext.train(sentences=train_data_rows, total_examples = len(train_data_rows), epochs=5)
            except Exception as e:
                print("Exception from spell model : ", str(e))
            err_fds=[]
            for col_fds in fds:
                if col_fds in err_fds_con:
                    continue
                else:
                    err_fds_con.append(col_fds)
                    err_fds.append(col_fds)
            if model_type=="Fasttext_G_F":
                error_correction=prepare_testing_datasets_real_world_data_error(dirty_data,clean_data,"fds",dataset_name,err_fds)
            else:
                error_correction=prepare_domain_testing_datasets_real_world_data_error(dirty_data,clean_data,"fds",dataset_name,domain_dirty_col,domain_clean_col,err_fds)
            for error_value, actual_value,want_to_clean,index in zip(error_correction['error'],error_correction['actual'],error_correction['want_to_clean'],error_correction['index']):
                try:
                    dirty_row=[]
                    if fds:
                        for fds_col in fds:
                            dirty_row.append(str(dirty_data.at[index, fds_col]))
                    if dirty_row:
                        #dirty_row = list(map(str, dirty_row))
                        dirty_row=list(filter(None, dirty_row))
                        if error_value:
                            dirty_row.remove(error_value)
                    #total_error=total_error+1
                    want_to_clean=str(want_to_clean)
                   # print(dirty_row, error_value)
                    if want_to_clean=="1" and len(error_value)<20 and total_error_to_repaired<10000:
                        total_error_to_repaired=total_error_to_repaired+1
                        if fds and dirty_row:
                            similar_value=model_fasttext.most_similar(positive=dirty_row, negative=[error_value])
                            #similar_value=model_fasttext.most_similar(dirty_row[0])
                            #R_Value=model.wv.most_similar(positive=dirty_row)
                        else:
                            similar_value=model_fasttext.most_similar(error_value)
                        #print(similar_value)
                        #R_Value=model.wv.most_similar(positive=dirty_row)
                        #print(R_Value)
                        #a=R_Value[0]
                        #b,c=a
                        #first=b
                        first,b=similar_value[0]
                        actual_value=str(actual_value)               
                        first=first.strip()
                        actual_value=actual_value.strip()
                        #print(first,actual_value)
                        if first==actual_value:
                            total_repaired=total_repaired+1
                    
                except Exception as e:
                    print('Error correction model: ',str(e))
            print(total_repaired,total_error_to_repaired)
        print(total_error,total_error_to_repaired,total_repaired )
        if total_error_to_repaired>0:
            p,r,f=evaluate_model(total_error,total_error_to_repaired,total_repaired)
        else:
            p,r,f="Invalid", "Invalid", "Invalid"
        return p,r,f

In [ ]:
def evaluate_model (total_error, total_error_to_repair, total_correction):
        if total_error_to_repair==0:
            precision=0.00
        else:
            precision=total_correction/total_error_to_repair
            precision=round(precision,2)
        if total_error==0:
            recall=0.00
        else:
            recall=total_correction/total_error
            recall=round(recall,2)
        if (precision+recall)==0:
            f_score=0.00
        else:
            f_score=(2 * precision * recall) / (precision + recall) 
            f_score=round(f_score,2)     
        return precision, recall,f_score